# CS 6120: Natural Language Processing
## Final Project: Music Genre Classification
## Authors: Bharat Chawla and Himaja R. Ginkala

In [15]:
# importing libraries
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
import string

pip install squarify

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

from keras.preprocessing.text import Tokenizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import warnings
warnings.filterwarnings('ignore')

SyntaxError: ignored

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# load data from CSV files ( upload them since our directory paths will be different for google drive location)
df_artists = pd.read_csv('/content/drive/My Drive/Graduate_School/CS_6120/NLP_Final_Project/artists-data.csv')
df_lyrics = pd.read_csv('/content/drive/My Drive/Graduate_School/CS_6120/NLP_Final_Project/lyrics-data.csv')

In [7]:
df_artists.head()

,Artist,Genres,Songs,Popularity,Link
0,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
1,Chiclete com Banana,Axé,268.0,3.8,/chiclete-com-banana/
2,Banda Eva,Axé; Romântico; Reggae,215.0,2.3,/banda-eva/
3,É O Tchan,Axé,129.0,1.6,/e-o-tchan/
4,Claudia Leitte,Pop; Axé; Romântico,167.0,1.5,/claudia-leitte/


In [8]:
df_lyrics.head()

,ALink,SName,SLink,Lyric,language
0,/ivete-sangalo/,Arerê,/ivete-sangalo/arere.html,"Tudo o que eu quero nessa vida,\nToda vida, é\...",pt
1,/ivete-sangalo/,Se Eu Não Te Amasse Tanto Assim,/ivete-sangalo/se-eu-nao-te-amasse-tanto-assim...,Meu coração\nSem direção\nVoando só por voar\n...,pt
2,/ivete-sangalo/,Céu da Boca,/ivete-sangalo/chupa-toda.html,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...,pt
3,/ivete-sangalo/,Quando A Chuva Passar,/ivete-sangalo/quando-a-chuva-passar.html,Quando a chuva passar\n\nPra quê falar\nSe voc...,pt
4,/ivete-sangalo/,Sorte Grande,/ivete-sangalo/sorte-grande.html,A minha sorte grande foi você cair do céu\nMin...,pt


### Data Exploration and Modification

In [9]:
# remove duplicates in artists data
df_artists = df_artists.drop_duplicates(subset = 'Link', keep ='first')

In [10]:
# rename columns in lyrics data
df_lyrics.rename(columns = {'ALink':'Link'}, inplace = True)
df_lyrics.rename(columns = {'language':'Language'}, inplace = True)

In [11]:
# merge both dataframes
df_artists_lyrics = pd.merge(df_artists, df_lyrics, on = 'Link')

# obtain final dataframe
df_custom = df_artists_lyrics[['Artist', 'SName', 'Genres', 'Lyric', 'Popularity', 'Language']]
df_final_data = df_custom[df_custom.Lyric.notnull()]

In [12]:
# create an english dataframe
df_english = df_final_data[df_final_data['Language'] == 'en']
df_english

,Artist,SName,Genres,Lyric,Popularity,Language
69,Ivete Sangalo,Careless Whisper,Pop; Axé; Romântico,I feel so unsure\nAs I take your hand and lead...,4.4,en
86,Ivete Sangalo,Could You Be Loved / Citação Musical do Rap: S...,Pop; Axé; Romântico,"Don't let them fool, ya\nOr even try to school...",4.4,en
88,Ivete Sangalo,Cruisin' (Part. Saulo),Pop; Axé; Romântico,"Baby, let's cruise, away from here\nDon't be c...",4.4,en
111,Ivete Sangalo,Easy,Pop; Axé; Romântico,"Know it sounds funny\nBut, I just can't stand ...",4.4,en
140,Ivete Sangalo,For Your Babies (The Voice cover),Pop; Axé; Romântico,You've got that look again\nThe one I hoped I ...,4.4,en
...,...,...,...,...,...,...
378983,Johnny Clegg,The Waiting,World Music; Gospel/Religioso,Chorus\nHere we stand waiting on the plain\nDa...,0.0,en
378984,Johnny Clegg,Too Early For The Sky,World Music; Gospel/Religioso,I nearly disappeared into the mouth of a croco...,0.0,en
378985,Johnny Clegg,Warsaw 1943 (I Never Betrayed The Revolution),World Music; Gospel/Religioso,"Amambuka, amambuka azothengisa izwe lakithi, i...",0.0,en
378986,Johnny Clegg,When The System Has Fallen,World Music; Gospel/Religioso,Sweat in the heat for days on end\nwaiting for...,0.0,en


In [13]:
print("Number of Languages: ", len(df_final_data['Language'].unique()))
print("Number of Artists: ", len(df_final_data['Artist'].unique()))

print("Number of English Artists: ", len(df_english['Artist'].unique()))
print("Number of English Songs: ", len(df_english['SName'].unique()))

Number of Languages:  52
Number of Artists:  4161
Number of English Artists:  2488
Number of English Songs:  137932


### Data Visualization

In [ ]:
# genre_counts = df_final_data['Genre'].value_counts()
# genre_names = df_final_data['Genre'].value_counts().index.tolist()

# # customize plot
# plt.figure(figsize = (7, 5))
# plt.title('Song Distribution by Genre in All Languages (Top 10)')
# plt.xticks(rotation='vertical')

# # plot pie chart
# plt.pie(genre_counts[0:10],
#         labels = genre_names[0:10],
#         startangle = 0,
#         autopct = '%1.0f%%',
#         pctdistance = 0.8,
#         colors = sns.color_palette('flare', 10),
#         wedgeprops = {"edgecolor" : "black",
#                       'linewidth': 0.5,
#                       'antialiased': True})
# plt.show()

In [ ]:
# genre_counts = df_english['Genre'].value_counts()
# genre_names = df_english['Genre'].value_counts().index.tolist()

# # customize plot
# plt.figure(figsize = (7, 5))
# plt.title('Song Distribution by Genre in English (Top 10)')
# plt.xticks(rotation='vertical')

# # plot pie chart
# plt.pie(genre_counts[0:10],
#         labels = genre_names[0:10],
#         startangle = 0,
#         autopct = '%1.0f%%',
#         pctdistance = 0.8,
#         colors = sns.color_palette('flare', 10),
#         wedgeprops = {"edgecolor" : "black",
#                       'linewidth': 0.5,
#                       'antialiased': True})
# plt.show()

In [ ]:
# genres_list = df_english['Genre'].unique()

# cleaned_genres_list = [x for x in genres_list]
# cleaned_genres_list.remove(np.nan)

# text = ""

# for w in cleaned_genres_list:
#   if " " in w:
#     separate = w.split(" ")
#     combined = separate[0] + "-" + separate[1]

#     text += " " + combined
#   else:
#     text += " " + w


# wordcloud = WordCloud(width = 1600, height = 1200, background_color = "white").generate(text)
# plt.figure(figsize = (12, 12), facecolor = None)
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.show()

### Data Preparation

In [ ]:
# global vocabulary
vocabulary = []

In [ ]:
def clean_lyric(lyric):
  # make all words lowercase
  lyric = lyric.lower()

  # remove links
  lyric = re.sub(r"<br /><br />", " ", lyric)

  # remove newline characters
  lyric = lyric.replace('\n', ' ')

  # remove non-alphanumeric characters
  pattern = "[^0-9a-zA-Z]+"
  lyric = re.sub(pattern, ' ', lyric)

  # remove punctuation
  lyric = lyric.translate(str.maketrans(' ', ' ', string.punctuation))

  # remove stopwords
  stop_words = set(stopwords.words('english'))
  lyric_tokens = word_tokenize(lyric)
  filtered_lyric = [w for w in lyric_tokens if w not in stop_words]

  # add any new words to global vocabulary
  for word in filtered_lyric:
    if word not in vocabulary:
      vocabulary.append(word)

  # return as joined string
  return " ".join(filtered_lyric)

In [ ]:
# create new column for cleaned lyrics
df_english['Cleaned_Lyric'] = np.nan

# reset indices of dataframe
df_english = df_english.reset_index(drop = True)

# clean lyrics in each row and assign to new column
for i, row in df_english.iterrows():
  df_english.at[i, 'Cleaned_Lyric'] = clean_lyric(df_english.at[i, 'Lyric'])

In [ ]:
# save dataframe to CSV file
df_english.to_csv('/content/drive/My Drive/Graduate_School/CS_6120/NLP_Final_Project/english_cleaned.csv', index = False)

# save vocabulary to TXT file
with open(r'/content/drive/My Drive/Graduate_School/CS_6120/NLP_Final_Project/vocabulary.txt', 'w') as f:
  for word in vocabulary:
    f.write('%s\n' % word)